In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
from model import C2W
from data import Char_LM_Dataset
from tqdm import tqdm
import config as argumentparser

In [3]:
import config as argumentparser
config = argumentparser.ArgumentParser()

In [5]:
if torch.cuda.is_available():
    torch.cuda.set_device(config.gpu)

In [6]:
torch.cuda.is_available()

False

In [7]:
# 导入训练集
training_set = Char_LM_Dataset(mode="train")
training_iter = torch.utils.data.DataLoader(dataset=training_set,
                                            batch_size=config.batch_size*config.max_sentence_length,
                                            shuffle=False,
                                            num_workers=2)

0 36192
1000 36192
2000 36192
3000 36192
4000 36192
5000 36192
6000 36192
7000 36192
8000 36192
9000 36192
10000 36192
11000 36192
12000 36192
13000 36192
14000 36192
15000 36192
16000 36192
17000 36192
18000 36192
19000 36192
20000 36192
21000 36192
22000 36192
23000 36192
24000 36192
25000 36192
26000 36192
27000 36192
28000 36192
29000 36192
30000 36192
31000 36192
32000 36192
33000 36192
34000 36192
35000 36192
36000 36192
(3619200,) (3619200, 16) (3619200,)


In [8]:
# 导入验证集
valid_set = Char_LM_Dataset(mode="valid")
valid_iter = torch.utils.data.DataLoader(dataset=valid_set,
                                         batch_size=config.batch_size*config.max_sentence_length,
                                         shuffle=False,
                                         num_workers=0)

0 698
(69800,) (69800, 16) (69800,)


In [9]:
# 导入测试集
test_set = Char_LM_Dataset(mode="test")
test_iter = torch.utils.data.DataLoader(dataset=test_set,
                                        batch_size=32*100,
                                        shuffle=False,
                                        num_workers=0)

0 657
(65700,) (65700, 16) (65700,)


In [10]:
model = C2W(config)

In [11]:
if config.cuda and torch.cuda.is_available():
    model.cuda()

In [13]:
criterion = nn.CrossEntropyLoss(reduce=False)
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
loss  = -1

F:\Program Files\anconda\lib\site-packages\torch\nn\_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [14]:
def get_test_result(data_iter,data_set):
    # 生成测试结果
    model.eval()
    all_ppl = 0
    for data, label,weights in data_iter:
        if config.cuda and torch.cuda.is_available():
            data = data.cuda()
            label = label.cuda()
            weights = weights.cuda()
        else:
            data = torch.autograd.Variable(data).long()
        label = torch.autograd.Variable(label).squeeze()
        out = model(data)
        loss_now = criterion(out, autograd.Variable(label.long()))
        ppl = (loss_now * weights.float()).view([-1, config.max_sentence_length])
        ppl = torch.sum(ppl, dim=1) / torch.sum((weights.view([-1, config.max_sentence_length])) != 0, dim=1).float()
        ppl = torch.sum(torch.exp(ppl))
        all_ppl += ppl.data.item()
    return all_ppl*config.max_sentence_length/data_set.__len__()

In [15]:
for epoch in range(config.epoch):
    model.train()
    process_bar = tqdm(training_iter)
    for data, label,weights in process_bar:
        if config.cuda and torch.cuda.is_available():
            data = data.cuda()
            label = label.cuda()
            weights = weights.cuda()
        else:
            data = torch.autograd.Variable(data).long()
        label = torch.autograd.Variable(label).squeeze()
        out = model(data)
        loss_now = criterion(out, autograd.Variable(label.long()))
        ppl = (loss_now*weights.float()).view([-1,config.max_sentence_length]) # pad不计算loss
        ppl = torch.sum(ppl,dim=1)/torch.sum((weights.view([-1,config.max_sentence_length]))!=0,dim=1).float()
        ppl = torch.mean(torch.exp(ppl)) # 计算困惑度
        loss_now = torch.sum(loss_now*weights.float())/torch.sum(weights!=0)
        if loss==-1:
            loss = loss_now.data.item()
        else:
            loss = 0.95 * loss + 0.05 * loss_now.data.item() # loss平滑输出
        process_bar.set_postfix(loss=loss,ppl=ppl.data.item())
        process_bar.update()
        optimizer.zero_grad()
        loss_now.backward()
        optimizer.step()
    print ("Valid ppl is:",get_test_result(valid_iter,valid_set))
    print ("Test ppl is:",get_test_result(test_iter,valid_set))

100%|█████████████████████████████████████████████████████████████| 181/181 [59:36<00:00, 18.61s/it, loss=5.8, ppl=358]


Valid ppl is: 292.959770661712
Test ppl is: 247.18495086407594


  0%|                                                                                          | 0/181 [00:00<?, ?it/s]

KeyboardInterrupt: 